<a href="https://colab.research.google.com/github/jotapdiasgh/pos-mvp-sprint2/blob/main/MVP_Sprint2_JoaoPedroVieiraDias_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVP - **Machine Learning & Analytics**

**Autor:** João Pedro Vieira Dias

**Data:** 26/09/2025

**Matrícula:** 4052025000227

**Dataset:** [Bank Marketing](https://archive.ics.uci.edu/dataset/222/bank+marketing)

# ❗ DISCLAIMER ❗ INFORMAÇÃO IMPORTANTE PARA A CORREÇÃO DO MVP - SPRINT 2

Após validação da possibilidade no Discord, decidi continuar com o mesmo dataset e a parte de Análise Exploratória de Dados do projeto da Sprint 1 a fim de conseguir construir, do início ao fim, meu primeiro projeto de **ML** com **Aprendizado Supervisionado** para responder a um problema de **Classificação**.

# 1. Escopo, objetivo e definição do problema

Essa seção traz as disposições iniciais sobre o dataset **Bank Marketing**, que compila informações acerca de uma campanha de marketing, feita por um banco português com seus clientes, a fim de convencê-los a aderir a modalidade de depósito a prazo no âmbito de investimentos. Ele foi escolhido por trazer informações reais, incluir dados socioeconômicos (envolve parte do meu emprego atual) e permitir a aplicação de vários dos conceitos estudados nas Sprints 1 e 2.

## 1.1. Contexto do Problema e Tipo de Tarefa

Por possuir um conjunto de informações de entrada (socioeconômicas e relacionadas à forma da campanha) e de saída (se o cliente aderiu ou não ao investimento > dimensão *target* com valor binário *sim* ou *não*), nota-se que o presente dataset apresente um problema de **Classificação** com possibilidade de criação de um modelo de **ML** com **Aprendizado Supervisionado**.

## 1.2. Área de aplicação

O insumo para a construção do presente modelo de ML provém de um dataset em formato .csv, previamente selecionado e carregado no Github; dessa forma, considerando a fonte de dados estruturada em linhas e colunas, trata-se de uma aplicação via **Dados Tabulares**.

## 1.3. Valor para o negócio

Analisar o dataset desse projeto para a construção de um modelo minimamente preditivo enseja importantes ganhos para a instituição. Considerando que esse tipo de atividade é comum e consideravelmente corriqueira para uma instituição financeira; também, considerando que a predição representa ganhos em termos de eficiência e padronização da atividade:


*   Há redução de recursos necessários de várias naturezas: pessoas, tempo, infraestrutura;
*   Sendo essa uma atividade que impacta diretamente no core do negócio (funding para operações de crédito, que geralmente representa 60% a 70% das receitas de instituições financeiras), há maior conversão e conhecimento de quais públicos são essenciais para manutenção do negócio ¹;
*   Com menos recursos direcionados para essa atividade, a instituição pode direcionar esses mesmos recursos a outras áreas que necessitem de maior atenção;
*   A padronização das condições para maior conversão nessas campanhas representam uso mais eficaz das pessoas envolvidas ou até substituição por robôs, com ganho potencial ainda maior em termos de escalabilidade das ações desse tipo;
*   Também há possibilidade de padronização de atuação em outras campanhas de marketing (ex: campanhas de produtos agregados para construção de fidelidade, como seguros e consórcios)

*¹ Informações provenientes de fontes como: relatórios de supervisão e estatística do Banco de Portugal (BdP), relatórios e análises setoriais da Associação Portuguesa de Bancos (APB), análises de empresas de Rating e Consultoria e relatórios individuais anuais de instituições financeiras*

# 2. Importação das Bibliotecas e Configurações para Reprodutibilidade

Essa seção consolida todas as importações de bibliotecas necessárias para o desenvolvimento do projeto, desde a análise e visualização dos dados originais, aos passos de pré-processamento dos dados, construção e testagem para coleta de resultados do modelo; ademais, a setagem das seeds globais e a impressão de informações úteis à documentação e registros pertinentes.

In [ ]:
# importações sistêmicas
import warnings
import random, sys, time

# importações basilares do código e voltadas para artifícios visuais
# (principalmente na seção de AED)
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# importação para divisão do dataset entre treino e teste
from sklearn.model_selection import train_test_split

# importações de métricas que serão utilizadas para avaliar os modelos do
# projeto
from sklearn.metrics import (f1_score, roc_auc_score, precision_score,
                             recall_score, balanced_accuracy_score, confusion_matrix, classification_report)

# importações para construção de pipelines para pré-processamento
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

# importação para construção de baseline
from sklearn.dummy import DummyClassifier

# importações para construção de pipelines dos modelos com otimizações possíveis
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

# importações para otimização de hiperparâmetros dos algoritmos de ML
from lightgbm import early_stopping, log_evaluation
from sklearn.model_selection import GridSearchCV, StratifiedKFold

# apenas uma alteração em avisos para melhor legibilidade de determinadas saídas
warnings.filterwarnings("ignore")

# setando o número global de seeds para garantia de reprodutibilidade em diferentes execuções do código
# a escolha de 42 seeds foi aleatória (muito comum como referência)
SEED = 42
np.random.seed(SEED)
random.seed(SEED)

# boa prática para impressão de informações úteis à documentação e registros pertinentes
print("Versão Python:", sys.version.split()[0])
print("Seed global:", SEED)

# 3. Funções

Para melhor organização do projeto e coerência nas etapas de modelagem posteriores, utilizo essa seção para estabelecer uma única métrica de comparação dos modelos otimizados com o baseline.

In [ ]:
# função para comparar algumas métricas entre os modelos
# f1_score: como métrica principal de avaliação em função da natureza do dataset
# (desbalanceado), balanceia falsos positivos e falsos negativos
# auc_roc_score: define quão bem os modelos distinguem 'sim' e 'não', mas pode
# ser otimista demais para esse tipo de dataset
# precision: determina quantos 'sim' foram preditos corretamente; bom para
# orçamentos enxutos ou custo de contato alto (não descrito nesse problema)
# recall: valida quantos 'sim' foram preditos do total possível; bom para
# orçamentos maiores ou custo de contato baixo
# balanced_accuracy: balanceia com pesos ponderados o desbalanceamento presente
# nesse dataset e ajuda a determinar melhor o desempenho dos modelos
# em conjunto com f1)
# accuracy:
def comparar_modelos(lista_modelos, X_test, y_test):

    print("======= COMPARAÇÃO COMPLETA DE MODELOS =======")
    print("=" * 46)

    resultados = {}

    for nome, modelo in lista_modelos.items():
        try:
            # etapa de previsões
            y_pred = modelo.predict(X_test)
            y_proba = modelo.predict_proba(X_test)[:, 1]

            # definição das métricas
            metricas = {
                'f1': f1_score(y_test, y_pred),
                'auc_roc': roc_auc_score(y_test, y_proba),
                'precision': precision_score(y_test, y_pred, zero_division=0),
                'recall': recall_score(y_test, y_pred, zero_division=0),
                'balanced_accuracy': balanced_accuracy_score(y_test, y_pred)
            }

            resultados[nome] = metricas

            # print das métricas para cada modelo
            print(f"\n{nome}")
            print("-" * 30)
            for metric, value in metricas.items():
                print(f"{metric:<18}: {value:.4f}")

        # caso algum modelo encontre problemas no cálculo das métricas
        except Exception as e:
            print(f"❌ Erro no modelo {nome}: {e}")
            resultados[nome] = None

    # ranking por F1-Score (métrica principal)
    print("\n" + "=" * 46)
    print("========== 🏆 RANKING POR F1-SCORE ==========")
    print("=" * 46)

    modelos_validos = {k: v for k, v in resultados.items() if v is not None}
    ranking = sorted(modelos_validos.items(), key=lambda x: x[1]['f1'], reverse=True)

    for i, (nome, metricas) in enumerate(ranking, 1):
        print(f"{i}º {nome:<15}: F1 = {metricas['f1']:.4f}")

    return resultados

# 4. Carga de dados

Essa seção vai carregar os dados do dataset original

In [ ]:
# carregamento do dataset via url raw do github
url = 'https://raw.githubusercontent.com/jotapdiasgh/pos-mvp-sprint2/refs/heads/main/bank-full.csv'

In [ ]:
# guardando o dataset em um dataframe com a devida formatação para melhor legibilidade
df = pd.read_csv(url, sep=';')

## 4.1. Atributos do Dataset

O dataset Bank Marketing contém 45.211 amostras dispostas em 17 dimensões, a saber:

- ***age*** - idade do cliente
- ***job*** - emprego do cliente
- ***marital*** - estado civil do cliente
- ***education*** - escolaridade do cliente
- ***default*** - presença de inadimplência em crédito do cliente
- ***balance*** - salário médio anual do cliente
- ***housing*** - presença de financiamento habitacional
- ***loan*** - presença de empréstimo pessoal
- ***contact*** - tipo de contato com o cliente
- ***day*** - dia do mês do último contato com o cliente
- ***month*** - mês do último contato com o cliente
- ***duration*** - duração em segundos do último contato com o cliente
- ***campaign*** - número de contatos feitos com o cliente
- ***pdays*** - número de dias desde o último contato com o cliente na última campanha
- ***previous*** - número de contatos feitos com o cliente antes da campanha atual
- ***poutcome*** - resultado da última campanha
- ***y*** - resultado da campanha atual

# 5. Análise Exploratória de Dados

Essa seção abarca técnicas referentes à análise exploratória e visualização de dados sobre o dataset Bank Marketing, de modo a compreender como suas variáveis estão distribuídas, como se relacionam entre si e quais são as características mais importantes para suportar a etapa posterior de pré-processamento dos dados.

## 5.1. Total e tipo das instâncias

O dataset Bank Marketing possui 45.211 instâncias (observações). Das 17 dimensões (colunas), 7 são do tipo numérico (integer) e 10 são do tipo categórico.

In [ ]:
# informações de identificação do dataset
print(f"O dataset possui {df.shape[0]} instâncias e {df.shape[1]} dimensões")
print("\nTipos de dados por dimensão:\n")
print(df.info())

## 5.2. Primeiras linhas do dataset

In [ ]:
# primeiras linhas do dataset
df.head()

Já nos primeiros dados conseguimos observar que podem faltar informações sobre o tipo de contato (contact) e se houve ou não conversão na última campanha (poutcome); ademais, a validação dos dias desde o último contato na campanha anterior (pdays) pode prejudicar algumas análises mais a frente.

## 5.3. Estatísticas descritivas

Usando estatística descritiva, observamos as principais características de cada dimensão para uma análise inicial

In [ ]:
# apresentação das estatísticas descritivas do dataset
print("\nEstatísticas descritivas:")
df.describe(include='all')

**Dimensões numéricas**
- Os clientes contatados possuem uma média de 41 anos (mais novo = 18 / mais velho = 95), e a maior parte desses clientes tem idades variando entre 31 e 51 anos

- Os clientes contatados possuem uma média de renda anual média de 1.362 € (menor = -8.019 € / maior = 102.127 €), e a maior parte desses clientes tem renda anual média entre -1.682 € e 4.406 €

- Os clientes foram contatados majoritariamente no dia 16, e a maior parte desses clientes recebeu contatos entre os dias 8 e 24

- Os clientes contatados permaneceram em ligação, em média, por 258 segundos
(menor duração = 0 / maior duração = 4.918), e a maior parte desses clientes ficaram em ligação entre 1 e 515 segundos > 0 indica que as ligações não foram sequer completadas

- Os clientes tiveram 3 contatos em média (menor quantidade = 1 / maior quantidade = 63), e a maior parte desses clientes recebeu entre 0 e 6 contatos > 0 indica que as ligações não foram sequer completadas

- Os clientes ficaram sem contato desde a última campanha por 40 dias, em média (menor intervalo = -1 / maior intervalo = 871), e a maior parte desses clientes ficou sem contato entre -60 e 140 dias > os números negativos indicam que não foram contatados na última campanha

- Os clientes tiveram, antes da campanha em análise, 1 contato em média (menor quantidade = 0 / maior quantidade = 275), e a maior parte desses clientes recebeu entre -1 e 3 contatos > 0 e números negativos indicam que não houve contato registrado

**Dimensões categóricas**
- Os clientes contatados estão empregados, em sua maioria, em cargos operacionais (blue-collar): são 9.732 instâncias

- Os clientes contatados são, em sua maioria, casados: são 27.214 instâncias

- Os clientes contatados possuem, em sua maioria, escolaridade secundária (até os 18 anos, semelhante ao ensino médio no Brasil): são 23.202 instâncias

- Os clientes contatados estão, em sua maioria, com inadimplência em créditos: são 44.396 instâncias

- Os clientes contatados possuem, em sua maioria, financiamento habitacional: são 25.130 instâncias

- Os clientes contatados possuem, em sua maioria, empréstimo pessoal: são 37.967 instâncias

- Os clientes contatados foram, em sua maioria, contatados de alguma forma: são 29.285 instâncias

- Os clientes foram contatados, em sua maioria, em maio: são 13.766 instâncias

- Dos clientes contatados, em sua maioria, não há informação de conversão com sucesso ou não na última campanha: são 36.959 instâncias

- Dos clientes contatados, em sua maioria, não houve conversão com sucesso: são 39.922 instâncias

### 5.3.1. Variáveis qualitativas

Como identificado que a conversão da campanha é muito baixa (a instância "no" da dimensão "y" aparece 39.992 vezes > 88,46% do dataset), é importante mapear inicialmente como as variáveis qualitativas estão distribuídas nesse conjunto em função da conversão positiva ou negativa.

#### 5.3.1.1. Análise - tipo de emprego

In [ ]:
# ----- primeiro gráfico, sem relação com a variável target 'y'
# área para plotar o gráfico
plt.figure(figsize=(12,6))

# seleção da variável e configuração da plotagem
ax = df['job'].value_counts().sort_values(ascending=True).plot(kind='barh', color='#FFAE80')

# adição de rótulo nos dados
for container in ax.containers:
    ax.bar_label(container, label_type='edge', padding=3, color='#212738')

# configuração dos aspectos do gráfico
ax.set_title('Quantidade de clientes por tipo de emprego')
ax.set_ylabel('Tipo de emprego')
ax.set_xlabel('Quantidade de clientes')

# visualização da plotagem
plt.show()

Somente empregos operacionais (blue-collar) representam 21,53% do dataset. Se somarmos estes aos empregos de gestão (management) e técnicos (technician), teremos 59,25% do dataset.

In [ ]:
# ----- segundo gráfico, relacionando com a variável target 'y'
# área para plotar o gráfico
fig, ax = plt.subplots(figsize=(12, 6))

# agrupamento de dados por 'job' e 'y'
contagem_job_y = df.groupby(['job', 'y']).size().unstack()

# ordenação pela conversão predominante
contagem_job_y = contagem_job_y.sort_values('no')

# seleção do agrupamento e configuração da plotagem
contagem_job_y.plot(kind='barh', stacked=True, color=['#F17300', '#3E7CB1'], ax=ax)

# adição de rótulo nos dados e melhor ajuste
for i, container in enumerate(ax.containers):
    if i == 0:  # Primeira categoria (base)
        ax.bar_label(container,
                    label_type='center',
                    fontsize=8,
                    color='#212738')
    else:  # Segunda categoria (topo)
        ax.bar_label(container,
                    label_type='edge',
                    fontsize=8,
                    color='#212738')

# configuração dos aspectos do gráfico
plt.title('Quantidade de clientes por tipo de emprego x conversão à campanha')
plt.xlabel('Quantidade de clientes')
plt.ylabel('Tipo de emprego')
plt.legend(title='Conversão à campanha')

# visualização da plotagem
plt.show()

Quando relacionado à variável target 'y', vemos que há certa proporcionalidade em relação ao número total por tipo de emprego, porém, com algum distanciamento: vai de 71,32% do total (student > estudantes) a 92,73% do total (blue-collar > operacionais)

#### 5.3.1.2. Análise - estado civil

In [ ]:
# ----- primeiro gráfico, sem relação com a variável target 'y'
# área para plotar o gráfico
plt.figure(figsize=(14,6))

# seleção da variável e configuração da plotagem
ax = df['marital'].value_counts().sort_values(ascending=True).plot(kind='barh', color='#FFAE80')

# adição de rótulo nos dados
for container in ax.containers:
    ax.bar_label(container, label_type='edge', padding=3, color='#212738')

# configuração dos aspectos do gráfico
ax.set_title('Quantidade de clientes por estado civil')
ax.set_ylabel('Estado civil')
ax.set_xlabel('Quantidade de clientes')

# visualização da plotagem
plt.show()

Somente os clientes casados (married) respondem por 60,19% do dataset.

In [ ]:
# ----- segundo gráfico, relacionando com a variável target 'y'
# área para plotar o gráfico
fig, ax = plt.subplots(figsize=(16, 6))

# agrupamento de dados por 'job' e 'y'
contagem_job_y = df.groupby(['marital', 'y']).size().unstack()

# ordenação pela conversão predominante
contagem_job_y = contagem_job_y.sort_values('no')

# seleção do agrupamento e configuração da plotagem
contagem_job_y.plot(kind='barh', stacked=True, color=['#F17300', '#3E7CB1'], ax=ax)

# adição de rótulo nos dados e melhor ajuste
for i, container in enumerate(ax.containers):
    if i == 0:  # Primeira categoria (base)
        ax.bar_label(container,
                    label_type='center',
                    color='#212738')
    else:  # Segunda categoria (topo)
        ax.bar_label(container,
                    label_type='edge',
                    color='#212738')

# configuração dos aspectos do gráfico
plt.title('Quantidade de clientes por estado civil x conversão à campanha')
plt.xlabel('Quantidade de clientes')
plt.ylabel('Estado civil')
plt.legend(title='Conversão à campanha')

# visualização da plotagem
plt.show()

Quando relacionado à variável target 'y', também vemos que há certa proporcionalidade em relação ao número total por estado civil: vai de 85,05% do total (single > solteiros) a 89,88% do total (married > casados)

#### 5.3.1.3. Análise - escolaridade

In [ ]:
# ----- primeiro gráfico, sem relação com a variável target 'y'
# área para plotar o gráfico
plt.figure(figsize=(14,6))

# seleção da variável e configuração da plotagem
ax = df['education'].value_counts().sort_values(ascending=True).plot(kind='barh', color='#FFAE80')

# adição de rótulo nos dados
for container in ax.containers:
    ax.bar_label(container, label_type='edge', padding=3, color='#212738')

# configuração dos aspectos do gráfico
ax.set_title('Quantidade de clientes por escolaridade')
ax.set_ylabel('Escolaridade')
ax.set_xlabel('Quantidade de clientes')

# visualização da plotagem
plt.show()

Somente os clientes secundaristas (secondary > similar ao ensino médio no Brasil) respondem por 51,32% do dataset.

In [ ]:
# ----- segundo gráfico, relacionando com a variável target 'y'
# área para plotar o gráfico
fig, ax = plt.subplots(figsize=(16, 6))

# agrupamento de dados por 'job' e 'y'
contagem_job_y = df.groupby(['education', 'y']).size().unstack()

# ordenação pela conversão predominante
contagem_job_y = contagem_job_y.sort_values('no')

# seleção do agrupamento e configuração da plotagem
contagem_job_y.plot(kind='barh', stacked=True, color=['#F17300', '#3E7CB1'], ax=ax)

# adição de rótulo nos dados e melhor ajuste
for i, container in enumerate(ax.containers):
    if i == 0:  # Primeira categoria (base)
        ax.bar_label(container,
                    label_type='center',
                    color='#212738')
    else:  # Segunda categoria (topo)
        ax.bar_label(container,
                    label_type='edge',
                    color='#212738')

# configuração dos aspectos do gráfico
plt.title('Quantidade de clientes por escolaridade x conversão à campanha')
plt.xlabel('Quantidade de clientes')
plt.ylabel('Escolaridade')
plt.legend(title='Conversão à campanha')

# visualização da plotagem
plt.show()

Quando relacionado à variável target 'y', também vemos que há certa proporcionalidade em relação ao número total por estado civil: vai de 84,99% do total (tertiary > universitários) a 91,37% do total (primary > primários, similar ao ensino fundamental no Brasil)

#### 5.3.1.4. Análise - inadimplência

In [ ]:
# ----- primeiro gráfico, sem relação com a variável target 'y'
# área para plotar o gráfico
plt.figure(figsize=(14,6))

# seleção da variável e configuração da plotagem
ax = df['default'].value_counts().sort_values(ascending=True).plot(kind='barh', color='#FFAE80')

# adição de rótulo nos dados
for container in ax.containers:
    ax.bar_label(container, label_type='edge', padding=3, color='#212738')

# configuração dos aspectos do gráfico
ax.set_title('Quantidade de clientes por presença de inadimplência')
ax.set_ylabel('Presença de inadimplência')
ax.set_xlabel('Quantidade de clientes')

# visualização da plotagem
plt.show()

98,20% do dataset não apresenta qualquer inadimplência.

In [ ]:
# ----- segundo gráfico, relacionando com a variável target 'y'
# área para plotar o gráfico
fig, ax = plt.subplots(figsize=(16, 6))

# agrupamento de dados por 'job' e 'y'
contagem_job_y = df.groupby(['default', 'y']).size().unstack()

# ordenação pela conversão predominante
contagem_job_y = contagem_job_y.sort_values('no')

# seleção do agrupamento e configuração da plotagem
contagem_job_y.plot(kind='barh', color=['#F17300', '#3E7CB1'], ax=ax)

# adição de rótulo nos dados e melhor ajuste
for i, container in enumerate(ax.containers):
    if i == 0:  # Primeira categoria (base)
        ax.bar_label(container,
                    label_type='center',
                    color='#212738')
    else:  # Segunda categoria (topo)
        ax.bar_label(container,
                    label_type='edge',
                    color='#212738')

# configuração dos aspectos do gráfico
plt.title('Quantidade de clientes por presença de inadimplência x conversão à campanha')
plt.xlabel('Quantidade de clientes')
plt.ylabel('Presença de inadimplência')
plt.legend(title='Conversão à campanha')

# visualização da plotagem
plt.show()

Quando relacionado à variável target 'y', foi necessário usar colunas agrupadas em função da escala deixar a instância 'no' com pouca legibilidade. Nesse sentido, é observado que 88,20% dos adimplentes (no) não tiveram conversão à campanha; e como esperado, a maior parte dos inadimplentes não adere à conversão: 93,62% (yes)

#### 5.3.1.5. Análise - financiamento habitacional

In [ ]:
# ----- primeiro gráfico, sem relação com a variável target 'y'
# área para plotar o gráfico
plt.figure(figsize=(14,6))

# seleção da variável e configuração da plotagem
ax = df['housing'].value_counts().sort_values(ascending=True).plot(kind='barh', color='#FFAE80')

# adição de rótulo nos dados
for container in ax.containers:
    ax.bar_label(container, label_type='edge', padding=3, color='#212738')

# configuração dos aspectos do gráfico
ax.set_title('Quantidade de clientes por presença de financiamento habitacional')
ax.set_ylabel('Presença de financiamento habitacional')
ax.set_xlabel('Quantidade de clientes')

# visualização da plotagem
plt.show()

Os clientes com financiamento habitacional representam a maior parte do dataset: 55,59%



In [ ]:
# ----- segundo gráfico, relacionando com a variável target 'y'
# área para plotar o gráfico
fig, ax = plt.subplots(figsize=(16, 6))

# agrupamento de dados por 'job' e 'y'
contagem_job_y = df.groupby(['housing', 'y']).size().unstack()

# ordenação pela conversão predominante
contagem_job_y = contagem_job_y.sort_values('no')

# seleção do agrupamento e configuração da plotagem
contagem_job_y.plot(kind='barh', stacked=True, color=['#F17300', '#3E7CB1'], ax=ax)

# adição de rótulo nos dados e melhor ajuste
for i, container in enumerate(ax.containers):
    if i == 0:  # Primeira categoria (base)
        ax.bar_label(container,
                    label_type='center',
                    color='#212738')
    else:  # Segunda categoria (topo)
        ax.bar_label(container,
                    label_type='edge',
                    color='#212738')

# configuração dos aspectos do gráfico
plt.title('Quantidade de clientes por presença de financiamento habitacional x conversão à campanha')
plt.xlabel('Quantidade de clientes')
plt.ylabel('Presença de financiamento habitacional')
plt.legend(title='Conversão à campanha')

# visualização da plotagem
plt.show()

Quando relacionado à variável target 'y', há discrepância mínima a ser observada: 92,30% dos clientes com financiamento habitacional não aderiram à conversão; esse percentual cai para 83,30% quando não há financiamento habitacional

#### 5.3.1.6. Análise - empréstimo pessoal

In [ ]:
# ----- primeiro gráfico, sem relação com a variável target 'y'
# área para plotar o gráfico
plt.figure(figsize=(14,6))

# seleção da variável e configuração da plotagem
ax = df['loan'].value_counts().sort_values(ascending=True).plot(kind='barh', color='#FFAE80')

# adição de rótulo nos dados
for container in ax.containers:
    ax.bar_label(container, label_type='edge', padding=3, color='#212738')

# configuração dos aspectos do gráfico
ax.set_title('Quantidade de clientes por presença de empréstimo pessoal')
ax.set_ylabel('Presença de empréstimo pessoal')
ax.set_xlabel('Quantidade de clientes')

# visualização da plotagem
plt.show()

Os clientes sem empréstimo pessoal representam a maior parte do dataset: 83,98%


In [ ]:
# ----- segundo gráfico, relacionando com a variável target 'y'
# área para plotar o gráfico
fig, ax = plt.subplots(figsize=(16, 6))

# agrupamento de dados por 'job' e 'y'
contagem_job_y = df.groupby(['loan', 'y']).size().unstack()

# ordenação pela conversão predominante
contagem_job_y = contagem_job_y.sort_values('no')

# seleção do agrupamento e configuração da plotagem
contagem_job_y.plot(kind='barh', stacked=True, color=['#F17300', '#3E7CB1'], ax=ax)

# adição de rótulo nos dados e melhor ajuste
for i, container in enumerate(ax.containers):
    if i == 0:  # Primeira categoria (base)
        ax.bar_label(container,
                    label_type='center',
                    color='#212738')
    else:  # Segunda categoria (topo)
        ax.bar_label(container,
                    label_type='edge',
                    color='#212738')

# configuração dos aspectos do gráfico
plt.title('Quantidade de clientes por presença de empréstimo pessoal x conversão à campanha')
plt.xlabel('Quantidade de clientes')
plt.ylabel('Presença de empréstimo pessoal')
plt.legend(title='Conversão à campanha')

# visualização da plotagem
plt.show()

Quando relacionado à variável target 'y', o comportamento é semelhante ao observado nos clientes que possuem financiamento habitacional: 93,32% dos que têm empréstimo pessoal não aderem à conversão; esse percentual cai para 87,34% quando é verificado quem não tem empréstimo pessoal

#### 5.3.1.7. Análise - tipo de contato

In [ ]:
# ----- primeiro gráfico, sem relação com a variável target 'y'
# área para plotar o gráfico
plt.figure(figsize=(14,6))

# seleção da variável e configuração da plotagem
ax = df['contact'].value_counts().sort_values(ascending=True).plot(kind='barh', color='#FFAE80')

# adição de rótulo nos dados
for container in ax.containers:
    ax.bar_label(container, label_type='edge', padding=3, color='#212738')

# configuração dos aspectos do gráfico
ax.set_title('Quantidade de clientes por tipo de contato')
ax.set_ylabel('Tipo de contato')
ax.set_xlabel('Quantidade de clientes')

# visualização da plotagem
plt.show()

Nesse ponto da análise é encontrado o primeiro problema considerável do dataset: não é possível determinar o tipo de contato feito com 28,80% da base. Já a maior parte (64,77%) foi feita por celular.

In [ ]:
# ----- segundo gráfico, relacionando com a variável target 'y'
# área para plotar o gráfico
fig, ax = plt.subplots(figsize=(16, 6))

# agrupamento de dados por 'job' e 'y'
contagem_job_y = df.groupby(['contact', 'y']).size().unstack()

# ordenação pela conversão predominante
contagem_job_y = contagem_job_y.sort_values('no')

# seleção do agrupamento e configuração da plotagem
contagem_job_y.plot(kind='barh', stacked=True, color=['#F17300', '#3E7CB1'], ax=ax)

# adição de rótulo nos dados e melhor ajuste
for i, container in enumerate(ax.containers):
    if i == 0:  # Primeira categoria (base)
        ax.bar_label(container,
                    label_type='center',
                    color='#212738')
    else:  # Segunda categoria (topo)
        ax.bar_label(container,
                    label_type='edge',
                    color='#212738')

# configuração dos aspectos do gráfico
plt.title('Quantidade de clientes por tipo de contato x conversão à campanha')
plt.xlabel('Quantidade de clientes')
plt.ylabel('Tipo de contato')
plt.legend(title='Conversão à campanha')

# visualização da plotagem
plt.show()

Quando relacionado à variável target 'y', como destacado na visualização anterior, a instância 'unknown' (desconhecido) traz um problema para a análise: não é possível afirmar como foi feito o contato para quem não aderiu à campanha nessa faixa (95,93% > maior proporção desse recorte). Essa proporção cai para 86,58% para quem foi contatado por telefone, e para 85,08% para quem foi contatado por celular.

#### 5.3.1.8. Análise - mês do último contato

In [ ]:
# ----- primeiro gráfico, sem relação com a variável target 'y'
# área para plotar o gráfico
plt.figure(figsize=(14,6))

# seleção da variável e configuração da plotagem
ax = df['month'].value_counts().sort_values(ascending=True).plot(kind='barh', color='#FFAE80')

# adição de rótulo nos dados
for container in ax.containers:
    ax.bar_label(container, label_type='edge', padding=3, color='#212738')

# configuração dos aspectos do gráfico
ax.set_title('Quantidade de clientes por mês de último contato')
ax.set_ylabel('Mês de último contato')
ax.set_xlabel('Quantidade de clientes')

# visualização da plotagem
plt.show()

Os contatos parecem estar bem divididos quando separamos na metade: 58,76% foram feitos no 1º semestre, sendo 30,44% do total e 51,81% do semestre somente em maio.

In [ ]:
# ----- segundo gráfico, relacionando com a variável target 'y'
# área para plotar o gráfico
fig, ax = plt.subplots(figsize=(16, 6))

# agrupamento de dados por 'job' e 'y'
contagem_job_y = df.groupby(['month', 'y']).size().unstack()

# ordenação pela conversão predominante
contagem_job_y = contagem_job_y.sort_values('no')

# seleção do agrupamento e configuração da plotagem
contagem_job_y.plot(kind='barh', stacked=True, color=['#F17300', '#3E7CB1'], ax=ax)

# adição de rótulo nos dados e melhor ajuste
for i, container in enumerate(ax.containers):
    if i == 0:  # Primeira categoria (base)
        ax.bar_label(container,
                    label_type='center',
                    color='#212738')
    else:  # Segunda categoria (topo)
        ax.bar_label(container,
                    label_type='edge',
                    color='#212738')

# configuração dos aspectos do gráfico
plt.title('Quantidade de clientes por mês de último contato x conversão à campanha')
plt.xlabel('Quantidade de clientes')
plt.ylabel('Mês de último contato')
plt.legend(title='Conversão à campanha')

# visualização da plotagem
plt.show()

Quando relacionado à variável target 'y', é notada uma diferença grande das proporções dos clientes que não aderiram à conversão de acordo com o mês: ela vai de 48,01% em março a 93,28% em maio. As menores proporções (até < 60%) compreendem, além de março, os meses de dezembro (53,27%), setembro (53,54%) e outubro (56,23%); os percentuais já saltam para > 80% nos demais meses, indicando quais são os melhores períodos para se obter conversão positiva à campanha.

#### 5.3.1.9. Análise - resultado da última campanha

In [ ]:
# ----- primeiro gráfico, sem relação com a variável target 'y'
# área para plotar o gráfico
plt.figure(figsize=(14,6))

# seleção da variável e configuração da plotagem
ax = df['poutcome'].value_counts().sort_values(ascending=True).plot(kind='barh', color='#FFAE80')

# adição de rótulo nos dados
for container in ax.containers:
    ax.bar_label(container, label_type='edge', padding=3, color='#212738')

# configuração dos aspectos do gráfico
ax.set_title('Quantidade de clientes por resultado da última campanha')
ax.set_ylabel('Resultado da última campanha')
ax.set_xlabel('Quantidade de clientes')

# visualização da plotagem
plt.show()

Assim como na análise do tipo de contato, nesse recorte a informação desconhecida (unknown) apresenta um problema no sentido de comparar a eficiência dos esforços: 81,75% dos clientes contatados estão sem a informação.

In [ ]:
# ----- segundo gráfico, relacionando com a variável target 'y'
# área para plotar o gráfico
fig, ax = plt.subplots(figsize=(16, 6))

# agrupamento de dados por 'job' e 'y'
contagem_job_y = df.groupby(['poutcome', 'y']).size().unstack()

# ordenação pela conversão predominante
contagem_job_y = contagem_job_y.sort_values('no')

# seleção do agrupamento e configuração da plotagem
contagem_job_y.plot(kind='barh', stacked=True, color=['#F17300', '#3E7CB1'], ax=ax)

# adição de rótulo nos dados e melhor ajuste
for i, container in enumerate(ax.containers):
    if i == 0:  # Primeira categoria (base)
        ax.bar_label(container,
                    label_type='center',
                    color='#212738')
    else:  # Segunda categoria (topo)
        ax.bar_label(container,
                    label_type='edge',
                    color='#212738')

# configuração dos aspectos do gráfico
plt.title('Quantidade de clientes por resultado da última campanha x conversão à campanha')
plt.xlabel('Quantidade de clientes')
plt.ylabel('Resultado da última campanha')
plt.legend(title='Conversão à campanha')

# visualização da plotagem
plt.show()

O desconhecimento da informação na maior parte dos clientes contatados é um problema ainda maior quando se percebe que 64,73% dos clientes com conversão positiva na última campanha, também tiveram conversão positiva na campanha em análise. Além disso, a informação 'other' não agrega valor à análise pois não há descrição do que ela representa.

#### 5.3.1.10. Análise - resultado da campanha (variável target 'y')

In [ ]:
# área para plotar o gráfico
plt.figure(figsize=(14,6))

# seleção da variável e configuração da plotagem
ax = df['y'].value_counts().sort_values(ascending=False).plot(kind='bar', color='#FFAE80')

# adição de rótulo nos dados
for container in ax.containers:
    ax.bar_label(container, label_type='edge', padding=3, color='#212738')

# configuração dos aspectos do gráfico
ax.set_title('Quantidade de clientes por resultado da campanha')
ax.set_ylabel('Clientes')
ax.set_xlabel('Resultado da campanha')

# visualização da plotagem
plt.show()

Aqui fica evidenciada a hipótese formulada com base em "percepção" no início desse estudo: 11,70% da campanha teve conversão positiva

### 5.3.2. Variáveis quantitativas

Usando a mesma premissa na seção de análise das variáveis qualitativas, é importante entender como as variáveis quantitativas estão distribuídas para relacionar seu padrão com a conversão da campanha.

#### 5.3.2.1. Média

A primeira medida de interesse que pode ser explorada nesse dataset para alguns insights é a média.

In [ ]:
# média das dimensões numéricas
print("\nDesvio padrão:")
df.describe().loc['mean']

Em média, o dataset concentra informações em clientes na fase adulta (~41 anos), com renda anual média 1,57x superior ao salário mínimo de portugal (780 € > cerca de R$ 8.700,00), sendo necessários 3 contatos geralmente feitos no meio do mês e durando cerca de 4 minutos; esses clientes ficaram cerca de 40 dias sem contato desde a última campanha, que foi feito somente 1 contato.

##### 5.3.2.1.1. Média - relação com variável target (y)

O gráfico a seguir relaciona a média das variáveis numéricas com a conversão à campanha, trazendo alguns primeiros insights.

In [ ]:
# listagem das variáveis numéricas para o gráfico
variaveis_numericas = ['age', 'day', 'duration', 'campaign', 'pdays',
                       'previous', 'balance']

# cálculo das médias em função da variável target 'y'
medias_por_y = df.groupby('y')[variaveis_numericas].mean().reset_index()

# área para plotar o gráfico
plt.figure(figsize=(10, 6))

# pivotagem das médias calculadas para preparar as variáveis para plotar no gráfico
medias_mix = medias_por_y.melt(id_vars='y', var_name='Variável',
                               value_name='Média')

# criação do gráfico de barras
barplot = sns.barplot(data=medias_mix, x='Variável', y='Média', hue='y',
                      palette={'yes': '#3E7CB1', 'no': '#F17300'})

# configuração dos aspectos do gráfico
plt.title('Médias das variáveis numéricas por conversão à campanha')
plt.xlabel('Variável')
plt.ylabel('Média')
plt.legend(title='Conversão à campanha')
plt.tight_layout()

# adição de valores e configurando parâmetros visuais do gráfico
for p in barplot.patches:
    barplot.annotate(
        f"{p.get_height():.1f}",
        (p.get_x() + p.get_width() / 2., p.get_height()),
        ha='center',
        va='center',
        xytext=(0, 8),
        textcoords='offset points'
    )

# visualização da plotagem
plt.show()

##### Média - principais insights

Olhando para as médias, nota-se que os clientes que tiveram conversão à campanha (y = yes):
- Ficaram 2,43x mais tempo em ligação do que os clientes que não tiveram conversão;
- Ficaram 1,89x mais dias sem contato (desde a última campanha) do que os clientes que não tiveram conversão;
- Possuem renda anual média 1,38x maior do que os clientes que não tiveram conversão;

#### 5.3.2.2. Desvio padrão

A segunda medida de interesse que pode ser explorada nesse dataset para alguns insights é o desvio padrão, visando complementar o que foi explorado na seção da Média.

In [ ]:
# desvio padrão das dimensões numéricas
print("\nDesvio padrão:")
df.describe().loc['std']

Quando se valida o desvio padrão, é verificado que o dataset apresenta um público muito variado: as variações mais significativas giram em torno da renda anual média (chegando a valores negativos, indicando desde clientes com problemas financeiros a clientes em situação relativamente confortável), duração dos contatos (indo de contato não realizado ao dobro da média), contatos realizados (de nenhum contato realizado ao dobro da média) e dias desde o último contato (clientes sem nenhum contato e clientes com relacionamento mais próximo).

##### 5.3.2.2.1. Desvio padrão - relação com variável target (y)

O gráfico a seguir relaciona o desvio padrão das variáveis numéricas com a conversão à campanha, trazendo mais alguns insights.

In [ ]:
# listagem das dimensões numéricas para o gráfico
dimensoes_numericas = df.select_dtypes(include=['int64', 'float64']).columns

# cálculo das médias em função da variável target 'y'
desvpad_por_y = df.groupby('y')[dimensoes_numericas].std().reset_index()

# área para plotar o gráfico
plt.figure(figsize=(10, 6))

# pivotagem das médias calculadas para preparar as variáveis para plotar no gráfico
desvpad_mix = desvpad_por_y.melt(id_vars='y', var_name='Variável', value_name='Desvio padrão')

# criação do gráfico de barras
barplot = sns.barplot(
    data=desvpad_mix,
    x='Variável',
    y='Desvio padrão',
    hue='y',
    palette={'yes': '#3E7CB1', 'no': '#F17300'}
)

# configuração dos aspectos do gráfico
plt.title('Desvio padrão das variáveis numéricas por conversão à campanha')
plt.xlabel('Variável')
plt.ylabel('Desvio padrão')
plt.legend(title='Conversão à campanha')
plt.tight_layout()

# adição de valores e configurando parâmetros visuais do gráfico
for p in barplot.patches:
    barplot.annotate(
        f"{p.get_height():.1f}",
        (p.get_x() + p.get_width() / 2., p.get_height()),
        ha='center',
        va='center',
        xytext=(0, 8),
        textcoords='offset points'
    )

# plotando o gráfico
plt.show()

##### Desvio padrão - principais insights

Olhando para o desvio padrão, nota-se que os clientes que tiveram conversão à campanha (y = yes):
- Se distanciam 1,32x a média de idade do que os clientes que não tiveram conversão;
- Se distanciam 1,89x a média de duração da ligação do que os clientes que não tiveram conversão;
- Se distanciam 1,22x a média de dias sem contato (desde a última campanha) do que os clientes que não tiveram conversão;
- Se distanciam 1,18x a renda anual média do que os clientes que não tiveram conversão;

#### 5.3.2.3. Matriz de correlação

Outra visualização interessante, considerando que existem muitas dimensões numéricas, é entender como elas se relacionam através da matriz de correlação.

In [ ]:
# listagem das dimensões numéricas para o gráfico
dimensoes_numericas = df.select_dtypes(include=['int64', 'float64']).columns

# criação de um dataframe apenas com as dimensões numéricas
df_numericas = df[dimensoes_numericas]

# cálculo da matriz de correlação
matriz_corr = df_numericas.corr()

# Plotar a matriz de correlação com Seaborn
plt.figure(figsize=(12, 10))
sns.heatmap(matriz_corr, annot=True, cmap='coolwarm', center=0, fmt=".2f",
            linewidths=.5, cbar_kws={"shrink": .8})
plt.title('Matriz de Correlação das Dimensões Numéricas - Bank Marketing', pad=10)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

A matriz de correlação demonstra que as variáveis possuem seus próprios comportamentos isolados, sem influência entre si. A única exceção demonstra uma pequena correlação entre pdays e previous, o que pode ser explicado pela interpretação do grande número de contatos não realizados antes da campanha atual.

#### 5.3.2.4. Distribuição das variáveis

Para se ter melhor ideia de como os dados estão dispersos no dataset, é importante explorar tal abordagem através de visualizações de distribuição das variáveis, como histogramas e boxplots.

##### 5.3.2.4.1. Análise - idade dos clientes

In [ ]:
# área para plotar o gráfico
plt.figure(figsize=(14, 6))

# seleção da variável e configuração da plotagem
n, bins, patches = plt.hist(df['age'], bins=20, edgecolor='#F2F4F3', color='#FFAE80')

# adição de rótulos nos dados
for i in range(len(n)):
    if n[i] > 0:
        plt.text(bins[i] + (bins[i+1] - bins[i])/2, n[i],
                 str(int(n[i])),
                 ha='center',
                 va='bottom')

# configuração dos aspectos do gráfico
plt.title('Distribuição da idade dos clientes')
plt.xlabel('Idade dos clientes')
plt.ylabel('Frequência')

# visualização da plotagem
plt.show()

In [ ]:
# primeiro boxplot, sem relação com variável target 'y'
# área para plotar o gráfico
plt.figure(figsize=(8, 4))

# seleção da variável e configuração da plotagem
sns.boxplot(data=df['age'], color='#212738', width=0.6)

# criação de variáveis para destacar as estatísticas mais importantes
mediana_idade = df['age'].median()
media_idade = df['age'].mean()
q1_idade = df['age'].quantile(0.25)
q3_idade = df['age'].quantile(0.75)

# configuração das estatísticas mais importantes
plt.axhline(mediana_idade, color='#F17300', linestyle='-.',
            linewidth=1.2,label=f'Mediana: {mediana_idade:.0f} anos')
plt.axhline(media_idade, color='#86B0D5', linestyle='--', linewidth=1.2,
            label=f'Média: {media_idade:.1f} anos')
plt.axhline(q1_idade, linestyle='none', label=f'1º Quartil: {q1_idade:.0f} anos')
plt.axhline(q3_idade, linestyle='none', label=f'3º Quartil: {q3_idade:.0f} anos')
plt.legend()

# configuração dos aspectos do gráfico
plt.title('Distribuição da idade dos clientes')
plt.ylabel('Idade dos clientes')
plt.rcParams['grid.color'] = '#DEE3E0'

# visualização da plotagem
plt.show()

In [ ]:
# conjunto de 2 boxplots, separando por relação com a variável target 'y'
# área para plotar o gráfico
plt.figure(figsize=(12, 6))

# boxplot para y = 'no'
plt.subplot(1, 2, 1)

# seleção da variável e configuração da plotagem
sns.boxplot(data=df[df['y'] == 'no']['age'], color='#86B0D5', width=0.6)

# criação de variáveis para destacar as estatísticas mais importantes
idade_no = df[df['y'] == 'no']['age']
q1_no = idade_no.quantile(0.25)
q3_no = idade_no.quantile(0.75)

# configuração das estatísticas mais importantes
plt.axhline(idade_no.median(), color='#F17300', linestyle='-.', linewidth=1.2,
            label=f'Mediana: {idade_no.median():.0f} anos')
plt.axhline(idade_no.mean(), color='#212738', linestyle='--', linewidth=1.2,
            label=f'Média: {idade_no.mean():.1f} anos')
plt.axhline(q1_no, linestyle='none', label=f'1º Quartil: {q1_no:.0f} anos')
plt.axhline(q3_no, linestyle='none', label=f'3º Quartil: {q3_no:.0f} anos')
plt.legend()

# configuração dos aspectos do gráfico
plt.title('Distribuição de idade por conversão NEGATIVA')
plt.ylabel('Idade')
plt.xlabel('Clientes com conversão NEGATIVA')
plt.rcParams['grid.color'] = '#DEE3E0'

### -------------------

# boxplot para y = 'yes'
plt.subplot(1, 2, 2)

# seleção da variável e configuração da plotagem
sns.boxplot(data=df[df['y'] == 'yes']['age'], color='#4CB963', width=0.6)

# criação de variáveis para destacar as estatísticas mais importantes
idade_yes = df[df['y'] == 'yes']['age']
q1_yes = idade_yes.quantile(0.25)
q3_yes = idade_yes.quantile(0.75)

# Linhas e legendas
plt.axhline(idade_yes.median(), color='#F17300', linestyle='-.', linewidth=1.2,
            label=f'Mediana: {idade_yes.median():.0f} anos')
plt.axhline(idade_yes.mean(), color='#212738', linestyle='--', linewidth=1.2,
            label=f'Média: {idade_yes.mean():.1f} anos')
plt.axhline(q1_yes, linestyle='none', label=f'1º Quartil: {q1_yes:.0f} anos')
plt.axhline(q3_yes, linestyle='none', label=f'3º Quartil: {q3_yes:.0f} anos')
plt.legend()

# configuração dos aspectos do gráfico
plt.title('Distribuição de idade por conversão POSITIVA')
plt.ylabel('Idade')
plt.xlabel('Clientes com conversão POSITIVA')
plt.rcParams['grid.color'] = '#DEE3E0'

# ajuste de layout e visualização da plotagem
plt.tight_layout()
plt.show()

- A distribuição da idade dos clientes se mostra Assimétrica Positiva, unimodal, concentrada nas faixas 30-50 anos e demonstrando a demografia-alvo do banco.
- Com mediana e média muito próximas, a distribuição por idade se mostra balanceada, com 50% dos clientes possuindo idade entre 33 e 48 anos e poucos outliers distorcendo a média de forma quase nula.
- Ao separar a análise por conversão à campanha, os números possuem pouca variação

##### 5.3.2.4.2. Análise - renda anual média

In [ ]:
# área para plotar o gráfico
plt.figure(figsize=(14, 6))

# seleção da variável e configuração da plotagem
n, bins, patches = plt.hist(df['balance'], bins=50, edgecolor='#F2F4F3',
                            color='#FFAE80')

# adição de rótulos nos dados
for i in range(len(n)):
    if n[i] > 0:
        plt.text(bins[i] + (bins[i+1] - bins[i])/2, n[i],
                 str(int(n[i])),
                 ha='center',
                 va='bottom')

# configuração dos aspectos do gráfico
plt.title('Distribuição da renda anual média dos clientes')
plt.xlabel('Renda anual média')
plt.ylabel('Frequência')

# visualização da plotagem
plt.show()

In [ ]:
# área para plotar o gráfico
plt.figure(figsize=(8, 6))

# seleção da variável e configuração da plotagem
sns.boxplot(data=df['balance'], color='#212738', width=0.6)

# criação de variáveis para destacar as estatísticas mais importantes
mediana_renda = df['balance'].median()
media_renda = df['balance'].mean()
q1_renda = df['balance'].quantile(0.25)
q3_renda = df['balance'].quantile(0.75)

# variável de referência para ajuste de proporção do eixo y
iqr = q3_renda - q1_renda

# configuração das estatísticas mais importantes e ajuste de escala no eixo y
## escala do eixo y foi ajustada para melhor legibilidade em função de muitos outliers
plt.axhline(mediana_renda, color='#F17300', linestyle='-.', linewidth=1.2, label=f'Mediana: {mediana_renda:.2f} €')
plt.axhline(media_renda, color='#86B0D5', linestyle='--', linewidth=1.2, label=f'Média: {media_renda:.2f} €')
plt.axhline(q1_renda, linestyle='none', label=f'1º Quartil: {q1_renda:.2f} €')
plt.axhline(q3_renda, linestyle='none', label=f'3º Quartil: {q3_renda:.2f} €')
plt.ylim([q1_renda - 3*iqr, q3_renda + 3*iqr])
plt.legend()

# configuração dos aspectos do gráfico
plt.title('Distribuição da renda anual média')
plt.ylabel('Renda anual média dos clientes')
plt.rcParams['grid.color'] = '#DEE3E0'

# visualização da plotagem
plt.show()

In [ ]:
print(df['balance'].sort_values(ascending=True).head(28267))

- A distribuição da renda anual média dos clientes se mostra Assimétrica Positiva com cauda longa a direita, unimodal, com 62,55% das suas instâncias em até 787 € (partindo de -8.019 €).
- Com mediana e média muito distantes, a distribuição por renda anual média se mostra muito desbalanceada, com 50% dos clientes com renda anual média entre 72 € e 1428 €. Mesmo que com poucos outliers em quantidade (proporcionalmente ao total), os valores desses outliers distorcem significativamente a média.

##### 5.3.2.4.3. Análise - dia do mês do último contato

In [ ]:
# área para plotar o gráfico
plt.figure(figsize=(16, 6))

# seleção da variável e configuração da plotagem
n, bins, patches = plt.hist(df['day'], bins=30, edgecolor='#F2F4F3',
                            color='#FFAE80')

# adição de rótulos nos dados
for i in range(len(n)):
    if n[i] > 0:
        plt.text(bins[i] + (bins[i+1] - bins[i])/2, n[i],
                 str(int(n[i])),
                 ha='center',
                 va='bottom')

# configuração dos aspectos do gráfico
plt.title('Distribuição do último dia do mês de contato com o cliente')
plt.xlabel('Último dia do mês de contato')
plt.ylabel('Frequência')

# visualização da plotagem
plt.show()

In [ ]:
# área para plotar o gráfico
plt.figure(figsize=(8, 6))

# seleção da variável e configuração da plotagem
sns.boxplot(data=df['day'], color='#212738', width=0.6)

# criação de variáveis para destacar as estatísticas mais importantes
mediana_dia = df['day'].median()
media_dia = df['day'].mean()
q1_dia = df['day'].quantile(0.25)
q3_dia = df['day'].quantile(0.75)

# configuração das estatísticas mais importantes
plt.axhline(mediana_dia, color='#F17300', linestyle='-.', linewidth=1.2, label=f'Mediana: {mediana_dia:.0f} dias')
plt.axhline(media_dia, color='#86B0D5', linestyle='--', linewidth=1.2, label=f'Média: {media_dia:.0f} dias')
plt.axhline(q1_dia, linestyle='none', label=f'1º Quartil: {q1_dia:.0f} dias')
plt.axhline(q3_dia, linestyle='none', label=f'3º Quartil: {q3_dia:.0f} dias')
plt.legend()

# configuração dos aspectos do gráfico
plt.title('Distribuição do último dia do mês de contato com o cliente')
plt.ylabel('Último dia do mês de contato com o cliente')
plt.rcParams['grid.color'] = '#DEE3E0'

# visualização da plotagem
plt.show()

- A distribuição do último dia do mês de contato com o cliente se mostra Simétrica, unimodal, bem dispersa (com poucos contatos no início do mês e próximo aos dias 10 e 25)
- Com mediana e média iguais, a distribuição por idade se mostra balanceada, com 50% dos dias de contato entre os dias 8 e 21. Não existem outliers.

##### 5.3.2.4.4. Análise - duração em segundos do último contato

In [ ]:
# área para plotar o gráfico
plt.figure(figsize=(16, 6))

# seleção da variável e configuração da plotagem
n, bins, patches = plt.hist(df['duration'], bins=10, edgecolor='#F2F4F3',
                            color='#FFAE80')

# adição de rótulos nos dados
for i in range(len(n)):
    if n[i] > 0:
        plt.text(bins[i] + (bins[i+1] - bins[i])/2, n[i],
                 str(int(n[i])),
                 ha='center',
                 va='bottom')

# configuração dos aspectos do gráfico
plt.title('Distribuição da duração do último contato com o cliente (em segundos)')
plt.xlabel('Duração do último contato com o cliente (em segundos)')
plt.ylabel('Frequência')

# visualização da plotagem
plt.show()

In [ ]:
# área para plotar o gráfico
plt.figure(figsize=(8, 6))

# seleção da variável e configuração da plotagem
sns.boxplot(data=df['duration'], color='#212738', width=0.6)

# criação de variáveis para destacar as estatísticas mais importantes
mediana_duration = df['duration'].median()
media_duration = df['duration'].mean()
q1_duration = df['duration'].quantile(0.25)
q3_duration = df['duration'].quantile(0.75)

# variável de referência para ajuste de escala do eixo y
iqr = q3_duration - q1_duration

# configuração das estatísticas mais importantes e ajuste de escala no eixo y
## escala do eixo y foi ajustada para melhor legibilidade em função de muitos outliers
plt.axhline(mediana_duration, color='#F17300', linestyle='-.', linewidth=1.2, label=f'Mediana: {mediana_duration:.0f} segundos')
plt.axhline(media_duration, color='#86B0D5', linestyle='--', linewidth=1.2, label=f'Média: {media_duration:.0f} segundos')
plt.axhline(q1_duration, linestyle='none', label=f'1º Quartil: {q1_duration:.0f} segundos')
plt.axhline(q3_duration, linestyle='none', label=f'3º Quartil: {q3_duration:.0f} segundos')
plt.ylim([q1_duration - 3*iqr, q3_duration + 3*iqr])
plt.legend()

# configuração dos aspectos do gráfico
plt.title('Distribuição da duração do último contato com o cliente (em segundos)')
plt.ylabel('Duração do último contato com o cliente')
plt.rcParams['grid.color'] = '#DEE3E0'

# visualização da plotagem
plt.show()

- A distribuição da duração do último contato dos clientes se mostra Assimétrica Positiva, unimodal, concentrada nas faixas 0 (ligação não completada) a 620 segundos, demonstrando que as ligações tendem a ser mais curtas.
- Com média acima da mediana, a distribuição por duração do último contato dos clientes se mostra desbalanceada, com 50% dos clientes permanecendo em ligação entre 103 e 319 segundos.
- Poucos outliers proporcionalmente ao número total.

*** OBS**: para essa dimensão específica, cabe destacar a descrição que o próprio autor do dataset inseriu (traduzido):
*"[...] este atributo afeta consideravelmente o resultado de saída (por exemplo, se duração=0, então y='não'). No entanto, a duração não é conhecida antes de um contato ser realizado. Além disso, após o término do contato, y é obviamente conhecido. Assim, este input só deve ser incluído para fins de benchmark e deve ser descartado se a intenção for ter um modelo preditivo realista."*

##### 5.3.2.4.5. Análise - contatos feitos

In [ ]:
# área para plotar o gráfico
plt.figure(figsize=(16, 6))

# seleção da variável e configuração da plotagem
n, bins, patches = plt.hist(df['campaign'], bins=10, edgecolor='#F2F4F3',
                            color='#FFAE80')

# adição de rótulos nos dados
for i in range(len(n)):
    if n[i] > 0:
        plt.text(bins[i] + (bins[i+1] - bins[i])/2, n[i],
                 str(int(n[i])),
                 ha='center',
                 va='bottom')

# configuração dos aspectos do gráfico
plt.title('Distribuição de contatos feitos com o cliente')
plt.xlabel('Contatos feitos com o cliente')
plt.ylabel('Frequência')

# visualização da plotagem
plt.show()

Como o histograma traz poucas informações (a distribuição fica muito concentrada em uma faixa que é importante analisar melhor), entendo ser necessário usar um gráfico de barras simples para destacar melhor a distribuição entre 1 e 8 contatos

In [ ]:
# área para plotar o gráfico
plt.figure(figsize=(14,8))

# seleção da variável e configuração da plotagem
ax = df['campaign'].value_counts().sort_values(ascending=False).plot(kind='bar', color='#FFAE80')

# adição de rótulo nos dados
for container in ax.containers:
    ax.bar_label(container, label_type='edge', padding=3, color='#212738')

# configuração dos aspectos do gráfico (com ênfase na distribuição mostrada no histograma anterior)
ax.set_title('Quantidade de contatos feitos com o cliente')
ax.set_ylabel('Clientes')
ax.set_xlabel('Quantidade de contatos')
ax.set_xlim(-1,7.5)

# visualização da plotagem
plt.show()

In [ ]:
# área para plotar o gráfico
plt.figure(figsize=(8, 6))

# seleção da variável e configuração da plotagem
sns.boxplot(data=df['campaign'], color='#212738', width=0.6)

# criação de variáveis para destacar as estatísticas mais importantes
mediana_campaign = df['campaign'].median()
media_campaign = df['campaign'].mean()
q1_campaign = df['campaign'].quantile(0.25)
q3_campaign = df['campaign'].quantile(0.75)

# variável de referência para ajuste de escala do eixo y
iqr = q3_campaign - q1_campaign

# configuração das estatísticas mais importantes e ajuste de escala no eixo y
## escala do eixo y foi ajustada para melhor legibilidade em função de muitos outliers
plt.axhline(mediana_campaign, color='#F17300', linestyle='-.', linewidth=1.2, label=f'Mediana: {mediana_campaign:.0f} contatos')
plt.axhline(media_campaign, color='#86B0D5', linestyle='--', linewidth=1.2, label=f'Média: {media_campaign:.0f} contatos')
plt.axhline(q1_campaign, linestyle='none', label=f'1º Quartil: {q1_campaign:.0f} contatos')
plt.axhline(q3_campaign, linestyle='none', label=f'3º Quartil: {q3_campaign:.0f} contatos')
plt.ylim([q1_campaign - 3*iqr, q3_campaign + 3*iqr])
plt.legend()

# configuração dos aspectos do gráfico
plt.title('Distribuição de contatos feitos com o cliente')
plt.ylabel('Contatos feitos com o cliente')
plt.rcParams['grid.color'] = '#DEE3E0'

# visualização da plotagem
plt.show()

- A distribuição da contatos feitos com o cliente se mostra Assimétrica Positiva, unimodal, concentrada nas faixas de 1 a 8 contatos, com destaque claro para clientes com 1 ou 2 contatos.
- Com média levemente acima da mediana, a distribuição por duração do último contato dos clientes se mostra desbalanceada, com cerca de 66,46% do total de contatos realizados concentrados entre 1 e 2 contatos.
- Poucos outliers proporcionalmente ao número total.

##### 5.3.2.4.6. Análise - dias desde o último contato na última campanha

In [ ]:
# área para plotar o gráfico
plt.figure(figsize=(16, 6))

# seleção da variável e configuração da plotagem
n, bins, patches = plt.hist(df['pdays'], bins=10, edgecolor='#F2F4F3',
                            color='#FFAE80')

# adição de rótulos nos dados
for i in range(len(n)):
    if n[i] > 0:
        plt.text(bins[i] + (bins[i+1] - bins[i])/2, n[i],
                 str(int(n[i])),
                 ha='center',
                 va='bottom')

# configuração dos aspectos do gráfico
plt.title('Distribuição de dias desde o último contato com o cliente na última campanha')
plt.xlabel('Dias desde o último contato com o cliente na última campanha')
plt.ylabel('Frequência')

# visualização da plotagem
plt.show()

Assim como na análise de contatos, o histograma traz poucas informações (a distribuição fica muito concentrada em uma faixa que é importante analisar melhor), sendo necessário usar um gráfico de barras simples para destacar melhor a distribuição entre -1 e 86 dias

In [ ]:
# área para plotar o gráfico
plt.figure(figsize=(14,8))

# seleção da variável e configuração da plotagem
ax = df['pdays'].value_counts().sort_values(ascending=False).plot(kind='bar', color='#FFAE80')

# adição de rótulo nos dados
for container in ax.containers:
    ax.bar_label(container, label_type='edge', padding=3, color='#212738')

# configuração dos aspectos do gráfico (com ênfase na distribuição mostrada no histograma anterior)
ax.set_title('Quantidade de dias desde o último contato com o cliente na última campanha')
ax.set_ylabel('Clientes')
ax.set_xlabel('Quantidade de dias')
ax.set_xlim(-1,7.5)

# visualização da plotagem
plt.show()

Nessa visão, notamos ausência de contatos (por quaisquer motivos) feitos na última campanha em 81,74% da base.

In [ ]:
# área para plotar o gráfico
plt.figure(figsize=(8, 6))

# seleção da variável e configuração da plotagem
sns.boxplot(data=df['pdays'], color='#212738', width=0.6)

# criação de variáveis para destacar as estatísticas mais importantes
mediana_diap = df['pdays'].median()
media_diap = df['pdays'].mean()
q1_diap = df['pdays'].quantile(0.25)
q3_diap = df['pdays'].quantile(0.75)

# configuração das estatísticas mais importantes
plt.axhline(mediana_diap, color='#F17300', linestyle='-.', linewidth=1.2, label=f'Mediana: {mediana_diap:.0f} dias')
plt.axhline(media_diap, color='#86B0D5', linestyle='--', linewidth=1.2, label=f'Média: {media_diap:.0f} dias')
plt.axhline(q1_diap, linestyle='none', label=f'1º Quartil: {q1_diap:.0f} dias')
plt.axhline(q3_diap, linestyle='none', label=f'3º Quartil: {q3_diap:.0f} dias')
plt.legend()

# configuração dos aspectos do gráfico
plt.title('Distribuição do último dia do mês de contato com o cliente')
plt.ylabel('Último dia do mês de contato com o cliente')
plt.rcParams['grid.color'] = '#DEE3E0'

# visualização da plotagem
plt.show()

Levando em consideração o contexto dessa dimensão (>81% da base sem registro de contatos), praticamente todos os demais valores se tornam outliers, o que inviabiliza a análise de insights.

##### 5.3.2.4.7. Análise - contatos feitos antes da campanha atual

In [ ]:
# área para plotar o gráfico
plt.figure(figsize=(14, 6))

# seleção da variável e configuração da plotagem
n, bins, patches = plt.hist(df['previous'], bins=10, edgecolor='#F2F4F3', color='#FFAE80')

# adição de rótulos nos dados
for i in range(len(n)):
    if n[i] > 0:
        plt.text(bins[i] + (bins[i+1] - bins[i])/2, n[i],
                 str(int(n[i])),
                 ha='center',
                 va='bottom')

# configuração dos aspectos do gráfico
plt.title('Distribuição de contatos feitos com o cliente antes da campanha atual')
plt.xlabel('Contatos feitos com o cliente antes da campanha atual')
plt.ylabel('Frequência')

# visualização da plotagem
plt.show()

Também para essa análise, o histograma traz poucas informações (a distribuição fica muito concentrada em uma faixa que é importante analisar melhor), sendo necessário usar um gráfico de barras simples para destacar melhor a distribuição entre 0 e 13 contatos

In [ ]:
# área para plotar o gráfico
plt.figure(figsize=(14,8))

# seleção da variável e configuração da plotagem
ax = df['previous'].value_counts().sort_values(ascending=False).plot(kind='bar', color='#FFAE80')

# adição de rótulo nos dados
for container in ax.containers:
    ax.bar_label(container, label_type='edge', padding=3, color='#212738')

# configuração dos aspectos do gráfico (com ênfase na distribuição mostrada no histograma anterior)
ax.set_title('Contatos feitos com o cliente antes da campanha atual')
ax.set_ylabel('Clientes')
ax.set_xlabel('Contatos feitos')
ax.set_xlim(-1,7.5)

# visualização da plotagem
plt.show()

Com forte relação com a análise anterior, temos o mesmo percentual com 0 contatos feitos antes da campanha atual: 81,74%

In [ ]:
# área para plotar o gráfico
plt.figure(figsize=(8, 8))

# seleção da variável e configuração da plotagem
sns.boxplot(data=df['previous'], color='#212738', width=0.6)

# criação de variáveis para destacar as estatísticas mais importantes
mediana_cont_ant = df['previous'].median()
media_cont_ant = df['previous'].mean()
q1_cont_ant = df['previous'].quantile(0.25)
q3_cont_ant = df['previous'].quantile(0.75)

# configuração das estatísticas mais importantes
plt.axhline(mediana_cont_ant, color='#F17300', linestyle='-.', linewidth=1.2, label=f'Mediana: {mediana_cont_ant:.0f} anos')
plt.axhline(media_cont_ant, color='#86B0D5', linestyle='--', linewidth=1.2, label=f'Média: {media_cont_ant:.1f} anos')
plt.axhline(q1_cont_ant, linestyle='none', label=f'1º Quartil: {q1_cont_ant:.0f} anos')
plt.axhline(q3_cont_ant, linestyle='none', label=f'3º Quartil: {q3_cont_ant:.0f} anos')
plt.legend()

# configuração dos aspectos do gráfico
plt.title('Distribuição contatos feitos com o cliente antes da campanha atual')
plt.ylabel('Contatos feitos com o cliente antes da campanha atual')
plt.rcParams['grid.color'] = '#DEE3E0'

# visualização da plotagem
plt.show()

Levando em consideração o contexto dessa dimensão (>81% da base sem registro de contatos), praticamente todos os demais valores se tornam outliers, o que inviabiliza a análise de insights.

# 6. Tratamento de dados e Pipeline de pré-processamento

Nessa etapa, os dados originais serão tratados de modo a serem redimensionados para melhor adequação à construção dos modelos em etapa posterior. Aqui, o desbalanceamento do dataset é tratado para não enviesar as análises e proporcionar maior qualidade na construção dos modelos; ademais, o pipeline de pré-processamento garante maior reprodutibilidade para replicação futura.

## 6.1. Separação em treino e teste geral

De início, é necessário separar o dataset em grupos de treino e teste para garantir avaliação e performance adequados dos modelos e diminuir a possibilidade de overfitting.

Também, é necessário usar o LabelEncoder para transformar a variável target y em binário antes dos pipelines.

Utilizei a mesma estrutura do template da Sprint 1 pois garante uma estrutura importante: prepara os dados para a construção dos modelos, separando os atributos (x) da variável target (y), e depois divide o dataset em treino (80%) e teste (20%).

In [ ]:
# separação das dimensões de entrada (x) e variável target (y) - holdout
X = df.drop('y', axis=1)
y = df['y']

# transformação da variável target y de yes/no para 1/0 usando LabelEncoder
le = LabelEncoder()
y_le = le.fit_transform(y)

# divisão dos dados em conjuntos de treino e teste na proporção 80/20 (convenção)
# stratify para correta divisão de 1/0 da variável target
X_train, X_test, y_train, y_test = train_test_split(X, y_le, test_size=0.2, random_state=SEED, stratify=y_le)

# validação da distribuição da variável target y para entendimento do nivel de
# desbalanceamento e devido tratamento posterior
print("Diagnóstico preliminar:\n")
print(f"Instâncias e dimensões de X_train: {X_train.shape}")
print(f"Distribuição de y_train: {np.unique(y_train, return_counts=True)}")
print(f"Proporção de classe 1 (yes): {y_train.mean()*100:.2f}%")

## 6.2. Pipelines para pré-processamento - transformação de dimensões

As dimensões numéricas e categóricas que necessitam de algum tipo de adequação serão transformadas e configuradas em pipelines utilizando OneHotEncoder, OrdinalEncoder e StandardScaler. Essas transformações, então, serão endereçadas em um único pipeline de pré-processamento.

In [ ]:
# identificação de dimensões numéricas e categóricas
cols_numericas = ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']
cols_cat_nom = ['job', 'marital', 'default', 'housing', 'loan', 'contact', 'poutcome']
cols_cat_ord = ['education', 'month']

# pipeline para dimensões numéricas com padronização (mais indicado para tratar
# os outliers do dataset), usando mediana como substituto para possíveis dados
# ausentes
transformador_num = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# pipeline para dimensões categóricas nominais com OneHotEncoder (mais indicado
# para tratar as dimensões que não possuem ordem natural), usando valor
# desconhecido no caso de dados ausentes; nesse caso, mesmo que as dimensões
# 'contact' e 'poutcome' possuam consideravelmente dados ausentes, determinei
# sparse_output=false pelo tamanho do dataset (< 100 mil instâncias)
transformador_cat_nom = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='unknown')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# pipeline para dimensões categóricas ordinais com OrdinalEncoder (mais indicado
# para tratar as dimensões que possuem ordem natural), usando valor
# desconhecido no caso de dados ausentes
transformador_cat_ord = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='unknown')),
    ('label', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

# combinação de todos os transformadores em um único pré-processador agregador
# de pipelines
preprocessador = ColumnTransformer(
    transformers=[
        ('num', transformador_num, cols_numericas),
        ('cat_nom', transformador_cat_nom, cols_cat_nom),
        ('cat_ord', transformador_cat_ord, cols_cat_ord)
    ])

## 6.3. Pipeline para definição e treinamento do baseline

Com os tratamentos devidos já realizados nas dimensões do dataset, é crucial que o projeto possua uma etapa de baseline para estabelecer os resultados basilares de modelos sem otimização, ou seja, valores de referência para comparação com os modelos otimizados em etapas posteriores. Aqui, utilizo o DummyClassifier com o pré-processador da etapa anterior.

Depois, realizo um rápido teste para validar a performance do baseline que será a âncora de viabilidade ou não de otimizações posteriores.

In [ ]:
# pipeline que integra o preprocessador com o DummyClassifier
baseline_dc = Pipeline(steps=[
    ('preprocessor', preprocessador),
    ('classifier', DummyClassifier(strategy='stratified', random_state=SEED))
])

# treinamento e teste rápido do baseline para posterior comparação concreta com
# os modelos otimizados
baseline_dc.fit(X_train, y_train)
y_pred_baseline = baseline_dc.predict(X_test)
print(f"F1-Score Baseline: {f1_score(y_test, y_pred_baseline):.4f}")
print(f"Previsões únicas do baseline: {np.unique(y_pred_baseline)}")

## 6.4. Pipelines para construção dos modelos - ensembles

Ajustado o pré-processamento, a construção dos modelos será feita via pipelines acoplando o pré-processador da etapa anterior. Os ensembles de boosting LGBM e XGBoost foram escolhidos para essa construção por possuirem alta performance em datasets como o desse projeto; segundo pesquisa própria, ambos são altamente usados hoje e conseguem performar bem com uso relativamente baixo de processamento.

### 6.4.1 Ensemble 1 - LGBM

O primeiro modelo descrito será o LGBM, referência para esse projeto em virtude dos seus altos ganhos sob a necessidade baixa de recursos para execução, além do seu desenho atender perfeitamente o tipo de problema proposto pelo dataset.

In [ ]:
# pipeline que integra o preprocessador com o LGBM
# esse modelo usa class_weight=balanced para tratar do desbalanceamento do
# dataset, n_estimators alto para otimização posterior, metric=binary_logloss
# para melhor avaliação na etapa de treinamento, boosting_type padrão e
# verbosity=0 para melhor legibilidade do console
modelo_lgbm = Pipeline(steps=[
    ('preprocessor', preprocessador),
    ('classifier', LGBMClassifier(
        random_state=SEED,
        class_weight='balanced',
        n_estimators=10000,
        metric='binary_logloss',
        boosting_type='gbdt',
        verbose=-1
    ))
])

### 6.4.2. Ensemble 2 - XGBoost

O segundo modelo descrito será o XGBoost, também referência para esse projeto em virtude mesmos pontos positivos do LGBM. Ele será utilizado para efeitos de comparação de performance.

In [ ]:
# pipeline que integra o preprocessador com o XGBoost
# esse modelo usa n_estimators alto para otimização posterior, eval_metric=logloss
# para melhor avaliação na etapa de treinamento, label_encoder=False para evitar
# warnings e verbosity=0 para melhor legibilidade do console
modelo_xgb = Pipeline(steps=[
    ('preprocessor', preprocessador),
    ('classifier', XGBClassifier(
        random_state=SEED,
        n_estimators=10000,
        eval_metric='logloss',
        early_stopping_rounds=50,
        use_label_encoder=False,
        verbosity=0
    ))
])

# 7. Otimização de hiperparâmetros com cross-validation e folds

Nessa seção, utilizo GridSearchCV para testar todas as possibilidades setadas em um grid e identificar os melhores hiperparâmetros possíveis (já aplicando o conceito de validação cruzada e separando os dados em folds via StratifiedKFold). Esse código utiliza um mecanismo de early stop para evitar que os pipelines de LGBM e XGBoost validem árvores desnecessárias e parem a execução no melhor n_estimator possível para boa performance.

## 7.1. Separação em treino e teste para otimização

Para essa etapa, é importante separar parte do conjunto de treino/teste (X/Y já definidos anteriormente) para aumentar a segurança na performance e prevenção de data leakage e overfitting. Aqui, o código usa parte das bases para buscar os melhores hiperparâmetros possíveis a serem utilizados nos modelos otimizados.

In [ ]:
# utilizo parte do treino/teste (15% de X/Y) para criar um mecanismo de early stop
# que garanta o melhor número possível de iterações nos modelos
X_fit, X_val, y_fit, y_val = train_test_split(X_train, y_train, test_size=0.15, random_state=SEED, stratify=y_train)

# ajuste nos dados de treino para otimizações posteriores e prevenção de
# data leakage
preprocessador.fit(X_fit, y_fit)
X_val_processed = preprocessador.transform(X_val)

## 7.2. Otimização para LGBM

Como algoritmo referência, a otimização é ajustada primeiro para o LGBM.

In [ ]:
# registro do momento inicial de execução desse bloco de código
start_time = time.time()

# configuração do CV com 5 folds para equilibrar performance e necessidade de
# capacidade computacional e shuffle=True para melhorar a aleatoriedade; setado
# separadamente para atender ao GridSearchCV
cv_lgbm = StratifiedKFold(n_splits=3, shuffle=True, random_state=SEED)

# definição dos parâmetros para o GridSearchCV baseados nas indicações da
# literatura dos algoritmos (LGBM e XGBoost) e no contexto do dataset do projeto
# onde max_depth balanceia árvores rasas e profundas de modo a evitar
# overfitting, learning_rate aceitável levando em conta o early stop e subsample
# para melhorar a possibilidade de aleatoriedade e diminuir overfitting
params_grid_lgbm = {
    'classifier__max_depth': [-1],
    'classifier__learning_rate': [0.05],
    'classifier__subsample': [0.8]
}

# configuração dos parâmetros de fit para o mecanismo de early stop, setando
# 50 iterações como máximo para identificar o ponto ideal, log_evaluation em -1
# para melhor legibilidade do log e binary_logloss para monitorar o desempenho
# da otimização
fit_params_lgbm = {
    'classifier__callbacks': [
        early_stopping(stopping_rounds=50),
        log_evaluation(-1)
    ],
    'classifier__eval_set': [(X_val_processed, y_val)],
    'classifier__eval_metric': 'binary_logloss',
}

# configuração do GridSearchCV para LGBM com scoring=f1 (ideal para dataset
# desbalanceado com foco em equilíbrio de dimensões majoritárias e minoritárias),
# n_jobs=-1 para usar todos os processadores disponíveis em cada fold e
# verbose=1 para acompanhar o progresso no log
grid_search_lgbm_fit = GridSearchCV(
    estimator=modelo_lgbm,
    param_grid=params_grid_lgbm,
    cv=cv_lgbm,
    scoring='f1',
    n_jobs=-1,
    verbose=1
)

# execução do GridSearchCV com os parâmetros necessários para early stop
grid_search_lgbm_fit.fit(X_fit, y_fit, **fit_params_lgbm)

# print dos melhores hiperparâmetros e score f1 após treino
print(f"\nMelhores parâmetros: {grid_search_lgbm_fit.best_params_}")
print(f"\nMelhor score (F1) de validação cruzada: {grid_search_lgbm_fit.best_score_:.4f}")

# registro do momento final de execução desse bloco de código e print da
# diferença para o registro do momento inicial; particularmente útil para os
# testes com os hiperparâmetros de otimização
execution_time = time.time() - start_time
print(f"Tempo de execução do GridSearch: {execution_time:.2f} segundos")

### 7.2.1. Log de testes e justificativa de hiperparâmetros escolhidos - LGBM

Essa seção compilará os testes realizados com n_splits diferentes a fim de justificar a escolha dos hiperparâmetros definidos nas seções de otimização. Esses testes levaram em conta o F1-Score obtido VERSUS tempo total de execução, onde procurei não prolongar além do necessário em virtude do tamanho do dataset.

#### Teste com n_splits = 2

parâmetros para o grid:

'classifier__max_depth': [5, 10, -1]

'classifier__learning_rate': [0.05, 0.1]

'classifier__subsample': [0.8, 1.0]


---


Fitting 2 folds for each of 12 candidates, totalling 24 fits

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1567]	valid_0's binary_logloss: 0.273362

Melhores parâmetros: {'classifier__learning_rate': 0.05, 'classifier__max_depth': 5, 'classifier__subsample': 0.8}

Melhor score (F1) de validação cruzada: **0.6054**

Tempo de execução do GridSearch: **49.66** segundos

#### Teste com n_splits = 3  |  ✅Melhor resultado (**F1-Score = 0.6142**)

parâmetros para o grid:

'classifier__max_depth': [5, 10, -1]

'classifier__learning_rate': [0.05, 0.1]

'classifier__subsample': [0.8, 1.0]


---


Fitting 3 folds for each of 12 candidates, totalling 36 fits

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1543]	valid_0's binary_logloss: 0.253268

Melhores parâmetros: {'classifier__learning_rate': 0.05, 'classifier__max_depth': -1, 'classifier__subsample': 0.8}

Melhor score (F1) de validação cruzada: **0.6142**

Tempo de execução do GridSearch: **74.25** segundos

#### Teste com n_splits = 4

parâmetros para o grid:

'classifier__max_depth': [5, 10, -1]

'classifier__learning_rate': [0.05, 0.1]

'classifier__subsample': [0.8, 1.0]


---


Fitting 4 folds for each of 12 candidates, totalling 48 fits

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1543]	valid_0's binary_logloss: 0.253268

Melhores parâmetros: {'classifier__learning_rate': 0.05, 'classifier__max_depth': -1, 'classifier__subsample': 0.8}

Melhor score (F1) de validação cruzada: **0.6129**

Tempo de execução do GridSearch: **112.95** segundos

#### Teste com n_splits = 5

parâmetros para o grid:

'classifier__max_depth': [5, 10, -1]

'classifier__learning_rate': [0.05, 0.1]

'classifier__subsample': [0.8, 1.0]


---


Fitting 5 folds for each of 12 candidates, totalling 60 fits

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[891]	valid_0's binary_logloss: 0.254204

Melhores parâmetros: {'classifier__learning_rate': 0.1, 'classifier__max_depth': 10, 'classifier__subsample': 0.8}

Melhor score (F1) de validação cruzada: **0.6111**

Tempo de execução do GridSearch: **140.70** segundos

## 7.3. Otimização para XGBoost

Para manter a equivalência de algoritmos, é feita a otimização do XGBoost

In [ ]:
# registro do momento inicial de execução desse bloco de código
start_time = time.time()

# configuração do CV com 3 folds para equilibrar performance e necessidade de
# capacidade computacional e shuffle=True para melhorar a aleatoriedade; setado
# separadamente para atender ao GridSearchCV de ambos: LGBM e XGBoost
cv_xgb = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)

# definição dos parâmetros para o GridSearchCV baseados nas indicações da
# literatura dos algoritmos (LGBM e XGBoost) e no contexto do dataset do projeto
# onde max_depth balanceia árvores rasas e profundas de modo a evitar
# overfitting, learning_rate aceitável levando em conta o early stop, subsample
# para melhorar a possibilidade de aleatoriedade e diminuir overfitting,
# colsample_bytree para otimizar generalização e os regs alpha e lambda (L1 e L2)
# para mais otimização em evitar overfitting
params_grid_xgb = {
    'classifier__max_depth': [7],
    'classifier__learning_rate': [0.1],
    'classifier__subsample': [0.8],
    'classifier__colsample_bytree': [0.9],
    'classifier__reg_alpha': [1],
    'classifier__reg_lambda': [1]
}

# configuração dos parâmetros de fit para o mecanismo de early stop, setando
# 50 iterações como máximo para identificar o ponto ideal e verbose = False
# para melhor legibilidade do log
fit_params_xgb = {
    'classifier__eval_set': [(X_val_processed, y_val)],
    'classifier__verbose': False
}

# configuração do GridSearchCV para XGB com scoring=f1 (ideal para dataset
# desbalanceado com foco em equilíbrio de dimensões majoritárias e minoritárias),
# n_jobs=1 para usar 1 processador em cada fold (validei que seria melhor para)
# esse algoritmo), verbose=1 para acompanhar o progresso no log e
# return_train_score=True para auxiliar na identificação de possível overfitting
# no log
grid_search_xgb_fit = GridSearchCV(
    estimator=modelo_xgb,
    param_grid=params_grid_xgb,
    cv=cv_xgb,
    scoring='f1',
    n_jobs=1,
    verbose=1,
    return_train_score=True
)

# execução do GridSearchCV com os parâmetros necessários para early stop
grid_search_xgb_fit.fit(X_fit, y_fit, **fit_params_xgb)

# print dos melhores hiperparâmetros e score f1 após treino
print(f"\nMelhores parâmetros: {grid_search_xgb_fit.best_params_}")
print(f"\nMelhor score (F1) de validação cruzada: {grid_search_xgb_fit.best_score_:.4f}")

# registro do momento final de execução desse bloco de código e print da
# diferença para o registro do momento inicial; particularmente útil para os
# testes com os hiperparâmetros de otimização
execution_time = time.time() - start_time
print(f"Tempo de execução do GridSearch: {execution_time:.2f} segundos")

### 7.3.1. Log de testes e justificativa de hiperparâmetros escolhidos - XGBoost

Essa seção compilará os testes realizados com n_splits diferentes a fim de justificar a escolha dos hiperparâmetros definidos nas seções de otimização. Esses testes levaram em conta o F1-Score obtido VERSUS tempo total de execução, onde procurei não prolongar além do necessário em virtude do tamanho do dataset.

#### Teste com n_splits = 2

parâmetros para o grid:

'classifier__max_depth': [3, 7]

'classifier__learning_rate': [0.05, 0.1]

'classifier__subsample': [0.8, 1.0]

'classifier__colsample_bytree': [0.9]

'classifier__reg_alpha': [0, 1]

'classifier__reg_lambda': [1]


---


Fitting 2 folds for each of 16 candidates, totalling 32 fits

Melhores parâmetros: {'classifier__colsample_bytree': 0.9, 'classifier__learning_rate': 0.1, 'classifier__max_depth': 7, 'classifier__reg_alpha': 0, 'classifier__reg_lambda': 1, 'classifier__subsample': 0.8}

Melhor score (F1) de validação cruzada: **0.5306**

Tempo de execução do GridSearch: **35.20** segundos

#### Teste com n_splits = 3

parâmetros para o grid:

'classifier__max_depth': [3, 7]

'classifier__learning_rate': [0.05, 0.1]

'classifier__subsample': [0.8, 1.0]

'classifier__colsample_bytree': [0.9]

'classifier__reg_alpha': [0, 1]

'classifier__reg_lambda': [1]


---


Fitting 3 folds for each of 16 candidates, totalling 48 fits

Melhores parâmetros: {'classifier__colsample_bytree': 0.9, 'classifier__learning_rate': 0.05, 'classifier__max_depth': 7, 'classifier__reg_alpha': 1, 'classifier__reg_lambda': 1, 'classifier__subsample': 0.8}

Melhor score (F1) de validação cruzada: **0.5450**

Tempo de execução do GridSearch: **67.84** segundos

#### Teste com n_splits = 4

parâmetros para o grid:

'classifier__max_depth': [3, 7]

'classifier__learning_rate': [0.05, 0.1]

'classifier__subsample': [0.8, 1.0]

'classifier__colsample_bytree': [0.9]

'classifier__reg_alpha': [0, 1]

'classifier__reg_lambda': [1]


---


Fitting 4 folds for each of 16 candidates, totalling 64 fits

Melhores parâmetros: {'classifier__colsample_bytree': 0.9, 'classifier__learning_rate': 0.05, 'classifier__max_depth': 7, 'classifier__reg_alpha': 0, 'classifier__reg_lambda': 1, 'classifier__subsample': 0.8}

Melhor score (F1) de validação cruzada: **0.5474**

Tempo de execução do GridSearch: **95.75** segundos

#### Teste com n_splits = 5  |  ✅Melhor resultado (**F1-Score = 0.5542**)

parâmetros para o grid:

'classifier__max_depth': [3, 7]

'classifier__learning_rate': [0.05, 0.1]

'classifier__subsample': [0.8, 1.0]

'classifier__colsample_bytree': [0.9]

'classifier__reg_alpha': [0, 1]

'classifier__reg_lambda': [1]


---


Fitting 5 folds for each of 16 candidates, totalling 80 fits

Melhores parâmetros: {'classifier__colsample_bytree': 0.9, 'classifier__learning_rate': 0.1, 'classifier__max_depth': 7, 'classifier__reg_alpha': 1, 'classifier__reg_lambda': 1, 'classifier__subsample': 0.8}

Melhor score (F1) de validação cruzada: **0.5542**

Tempo de execução do GridSearch: **125.04** segundos

#### Teste com n_splits = 6

parâmetros para o grid:

'classifier__max_depth': [3, 7]

'classifier__learning_rate': [0.05, 0.1]

'classifier__subsample': [0.8, 1.0]

'classifier__colsample_bytree': [0.9]

'classifier__reg_alpha': [0, 1]

'classifier__reg_lambda': [1]


---


Fitting 6 folds for each of 16 candidates, totalling 96 fits

Melhores parâmetros: {'classifier__colsample_bytree': 0.9, 'classifier__learning_rate': 0.05, 'classifier__max_depth': 7, 'classifier__reg_alpha': 0, 'classifier__reg_lambda': 1, 'classifier__subsample': 0.8}

Melhor score (F1) de validação cruzada: **0.5519**

Tempo de execução do GridSearch: **155.57** segundos

# 8. Teste dos modelos e resultados finais

Ajustados os modelos otimizados com seus devidos hiperparâmetros, é feita a comparação de suas performances em contraste com a performance do baseline usando F1-Score como métrica apropriada.

## 8.1. Comparação dos modelos via métricas

Aqui, os modelos escolhidos são comparados através das métrica estabelecidas no início do projeto com os (melhores) hiperparâmetros definidos na etapa do Gridsearch; junto com eles, o baseline para referência de melhora.

In [ ]:
# agregação dos 3 modelos em 1 variável a ser passada para a função de comparação
modelos_para_comparacao = {
    'Baseline': baseline_dc,
    'LightGBM': grid_search_lgbm_fit.best_estimator_,
    'XGBoost': grid_search_xgb_fit.best_estimator_
}

# execução da função de comparação de performance dos 3 modelos
resultados = comparar_modelos(modelos_para_comparacao, X_test, y_test)

## 8.2. Estratificação das métricas do melhor modelo

Por fim, são apresentadas as métricas do melhor modelo (segundo critérios estabelecidos) a fim de embasar e complementar as conclusões do projeto.

In [ ]:
# identificação do melhor modelo para posterior plotagem da matriz de confusão
melhor_modelo_nome = max(
    {k: v for k, v in resultados.items() if v is not None},
    key=lambda x: resultados[x]['f1']
)

# identificação do melhor modelo segundo métricas avaliadas
melhor_modelo = modelos_para_comparacao[melhor_modelo_nome]

# melhor y predito considerando X_test
y_pred_melhor = melhor_modelo.predict(X_test)

# print do melhor modelo
print(f"Melhor modelo: {melhor_modelo_nome} 🥇\n")

# compilado de métricas para validação da performance do melhor modelo,
# muito importante para dados desbalanceados
print(classification_report(y_test, y_pred_melhor, target_names=['No', 'Yes']))

# confusion matrix para visualização clara de positivos verdadeiros/falsos e
# negativos verdadeiros/falsos
cm = confusion_matrix(y_test, y_pred_melhor)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Matriz de Confusão')
plt.ylabel('Real')
plt.xlabel('Predito')
plt.show()

# 9. Conclusão

Conforme constatado na seção de Análise Exploratória de Dados, algumas das dimensões estão altamente desbalanceadas e precisaram de tratamento para que fossem adequadamente avaliados pelos modelos. Nesse sentido, após várias execuções do código, entendo que as etapas de pré-processamento e dos treinamentos com folds e cross-validation foram importantes para diminuir data leakage e proporcionar simetria de condições para que os modelos pudessem ser testados adequadamente, garantindo reprodutibilidade suficiente para futuro desenvolvimento.

## 9.1. Resultados encontrados

Após várias rodadas de execução do código, tanto na parte de otimização de hiperparâmetros dos modelos quanto do baseline, cheguei a números importantes das métricas utilizadas para comparação dos modelos:

*   **F1-Score**: em comparação à pontuação do baseline (0.1126), os modelos conseguiram alcançar números de 5 a 6x maiores (0.6010 para LightGBM e 0.5546 para XGBoost). Essas pontuações, segundo pesquisa de competições de top scores públicos no Kaggle, ficam relativamente próximas do SOTA de 0.648 para esse dataset.

*   **Auc-Roc**: em comparação à pontuação do baseline (0.4970), os modelos conseguiram alcançar números quase 2x maiores (0.9233 para LightGBM e 0.9338 para XGBoost), muito próximos de 1.

*   **Precision**: em comparação à pontuação do baseline (0.1115), os modelos conseguiram alcançar números de 5 a 6x maiores (0.5250 para LightGBM e 0.6711 para XGBoost). Destaque para XGB pela performance consideravelmente melhor que LGBM.

*   **Recall**: em comparação à pontuação do baseline (0.1068), os modelos conseguiram alcançar números de 4 a 7x maiores (0.7250 para LightGBM e 0.4726 para XGBoost). Ponto positivo para LGBM com performance substancialmente melhor que XGB.

*   **Balanced Accuracy**: em comparação à pontuação do baseline (0.4970), os modelos conseguiram alcançar números quase 2x maiores (0.8190 para LightGBM e 0.7210 para XGBoost). Mais um ponto pra LGBM.

Apesar de ser possível obter melhores resultados para as métricas, entendo que o trade-off com o tempo necessário de processamento não seria viável para o desenvolvimento desse projeto em específico; contudo, acredito que possa valer a pena em um ambiente real de aplicação no negócio.

Já com relação aos modelos escolhidos, entendo que tenham correspondido satisfatoriamente ao desafio proposto. Com parâmetros relativamente menos robustos, ambos foram capazes de superar muito o baseline a um custo baixo de processamento computacional.

Nesse sentido, é válido afirmar que, mesmo com preparações e otimizações de complexidade não tão alta, o projeto é viável do ponto de vista de negócio e pode ser aprimorado para que seja implementado, de fato.

## 9.2. Desafios do desenvolvimento

Durante o desenvolvimento desse projeto, tive alguns problemas maiores e que minaram sua progressão em diferentes níveis:

*   Seleção dos modelos que fariam a composição do projeto
*   Tempo hábil para deixar o projeto mais robusto possível, considerando o tempo disponível e o nível inicial de entendimento do tema tratado
*   Construção dos pipelines do baseline e dos modelos: dificuldade na seleção dos parâmetros mais indicados para esse tipo de problema
*   Modelos de otimização: dificuldade na seleção dos parâmetros mais indicados e no seu dimensionamento (quantidade e quais árvores em ambos os modelos, por exemplo)
*   Entendimento da importância de cada métrica para o tipo de problema e como validar a importância de cada uma
*   Construção da função para retorno da análise das métricas de performance dos modelos
*   Interpretação da matriz de confusão

## 9.3. Melhorias futuras

Através de pesquisas sobre melhores práticas, identiquei alguns pontos de melhorias futuras para aprimorar os modelos:

*   Utilizar mais análises voltadas ao tema do projeto, que possam fazer sentido em conjunto com as métricas avaliadas e trazer ganhos ao negócio, como Lift Score
*   Avaliar possibilidade de usar Feature Selection, como SelectKBest e RFE
*   Aumentar as combinações no gridsearch de ambos os modelos (o tempo de processamento - que estava chegando em quase 1h por execução do código - inviabilizou testes nesse sentido)
*   Testar o CatBoost para validar seus achados com os modelos presentes nesse projeto; em pesquisas, validei que também pode ser um bom algoritmo (tão bom quanto os 2 validados aqui)
*   Utilizar o conceito de Ensemble Híbrido para unir o melhor dos mundos, incluindo também Random Forest